In [1]:
import functions_py3 as fc

import json
jlo = lambda s: json.load(open(s,'r'))
%pylab inline
%config InlineBackend.figure_format = 'retina'
from itertools import combinations
import os
from scipy import interpolate
import collections as co
import gln_v8_parallel_py3 as gln

Populating the interactive namespace from numpy and matplotlib


In [29]:
worm = {}
with open('./fullanimal_05_02_2017_display2.sql','r') as f:
    for line in f:
        if line[0] == '(':
            a = line.strip().split(', ')
            se = a[10][1:-1]
            contin = a[5]
            obj1 = a[9][1:-1]
            obj2 = a[17][1:-1]
            x1,y1,z1 = a[0][1:],a[1],a[2]
            x2,y2,z2 = a[11],a[12],a[13]
            if se in worm:
                if contin not in worm[se]:
                    worm[se][contin] = {'0':{'end_points':[obj1,obj2],'points':[[x1,y1,z1],[x2,y2,z2]],'objs':[obj1,obj2]}}
                else:
                    if obj1 == worm[se][contin][str(len(worm[se][contin])-1)]['end_points'][1]:
                        worm[se][contin][str(len(worm[se][contin])-1)]['end_points'][1] = obj2
                        worm[se][contin][str(len(worm[se][contin])-1)]['points'].append([x2,y2,z2])
                        worm[se][contin][str(len(worm[se][contin])-1)]['objs'].append(obj2)
                    else:
                        worm[se][contin][str(len(worm[se][contin]))] = {'end_points':[obj1,obj2],'points':[[x1,y1,z1],[x2,y2,z2]],'objs':[obj1,obj2]}
                        
                    
            else:
                worm[se] = {contin:{'0':{'end_points':[obj1,obj2],'points':[[x1,y1,z1],[x2,y2,z2]],'objs':[obj1,obj2]}}}

In [75]:
## get rid of contin ##
NR = {}
for con in worm['NR']:
    for i in worm['NR'][con]:
        NR[str(len(NR))] = worm['NR'][con][i]

In [50]:
synapses = {}
with open('./fullanimal_05_02_2017_synapsecombined.sql','r') as f:
    for line in f:
        if line[0] == '(':
            a = line.strip().split(', ')
            se = a[11][1:-1]
            pre = a[14][1:-1]
            post = [a[15+i][1:-1] for i in range(int(a[12]))]
            if se not in synapses:
                synapses[se] = {'0':{'pre':pre, 'post':post}}
            else:
                synapses[se][str(len(synapses[se]))] = {'pre':pre, 'post':post}

In [58]:
obj2coor = {}
for i in NR:
    for j in range(len(NR[i]['objs'])):
        obj2coor[NR[i]['objs'][j]] = NR[i]['points'][j]
    

In [69]:
## incoorperate synapses ##
flag = 0
for i in synapses['NR']:
    pre = synapses['NR'][i]['pre']
    post = synapses['NR'][i]['post']
    try:
        pre_coor = obj2coor[pre]
        tmp_list = list(NR.keys())
        for j in tmp_list:
            if len(set(post).intersection(set(NR[j]['objs'])))>0:
                for k in post:
                    try:
                        idx = NR[j]['objs'].index(k)
                        NR[j]['points'][idx] = pre_coor
                        flag += 1
                        if idx < len(NR[j]['objs'])-1 and idx > 0:
                            NR[str(len(NR))] = {'end_points':[pre,NR[j]['end_points'][1]],
                                       'points':NR[j]['points'][idx:],
                                       'objs':NR[j]['objs'][idx:]}
                            NR[j]['end_points'][1] = pre
                            NR[j]['points'] = NR[j]['points'][:idx+1]
                            NR[j]['objs'] = NR[j]['objs'][:idx+1]
                        elif idx==0:
                            NR[j]['end_points'][0] = pre
                        elif idx == len(NR[j]['objs'])-1:
                            NR[j]['end_points'][-1] = pre
                    except ValueError:
                        continue
            elif pre in NR[j]['objs']:
                idx = NR[j]['objs'].index(pre)
                if idx < len(NR[j]['objs'])-1 and idx > 0:
                    NR[str(len(NR))] = {'end_points':[pre,NR[j]['end_points'][1]],
                                       'points':NR[j]['points'][idx:],
                                       'objs':NR[j]['objs'][idx:]}
                    NR[j]['end_points'][1] = pre
                    NR[j]['points'] = NR[j]['points'][:idx+1]
                    NR[j]['objs'] = NR[j]['objs'][:idx+1]
    except KeyError: ## some pre are not in obj2coor
        continue
                    

In [86]:
## delete twigs ##
# links_f = NR
nodes = array([links_f[j]['end_points'][0] for j in links_f]+[links_f[j]['end_points'][1] for j in links_f])
n_list = array(list(set(nodes)))
degrees = array([len(where(nodes==i)[0]) for i in n_list])
ends = n_list[where(degrees==1)[0]]
flag = 1
while flag > 0:
    tmp = list(links_f.keys())
    flag = 0
    for i in tmp:
        if links_f[i]['end_points'][0] in ends or links_f[i]['end_points'][1] in ends:
            del links_f[i]
            flag = 1
#             elif (links_f[i]['end_points'][0] in ends) and (links_f[i]['end_points'][1] in neuron_synapse_nodes[idx]):
#                 del links_f[i]
#                 flag = 1
#             elif (links_f[i]['end_points'][1] in ends) and (links_f[i]['end_points'][0] in neuron_synapse_nodes[idx]):
#                 del links_f[i]
#                 flag = 1
    nodes = array([links_f[j]['end_points'][0] for j in links_f]+[links_f[j]['end_points'][1] for j in links_f])
    n_list = array(list(set(nodes)))
    degrees = array([len(where(nodes==i)[0]) for i in n_list])
    ends = n_list[where(degrees==1)[0]]              
        

In [110]:
## clear degree-2 nodes ##
nodes = array([links_f[j]['end_points'][0] for j in links_f]+[links_f[j]['end_points'][1] for j in links_f])
n_list = array(list(set(nodes)))
# co = collections.Counter(nodes)
# degrees = array([co[node] for node in n_list])
degrees = array([len(where(nodes==i)[0]) for i in n_list])
targets = n_list[where(degrees==2)[0]]
while len(targets) > 0:
    for i in targets:
        pair = []
        for j in links_f:
            if links_f[j]['end_points'][0]==i or links_f[j]['end_points'][1]==i:
                pair.append(j)
        if len(pair) == 2:
            if links_f[pair[0]]['end_points'][0] == i:
                if links_f[pair[1]]['end_points'][0] == i:
                    links_f[pair[0]]['end_points'] = [links_f[pair[1]]['end_points'][1],links_f[pair[0]]['end_points'][1]]
                    links_f[pair[0]]['points'] = links_f[pair[1]]['points'][::-1][:-1] + links_f[pair[0]]['points']
                    links_f[pair[0]]['objs'] = links_f[pair[1]]['objs'][::-1][:-1] + links_f[pair[0]]['objs']
                    del links_f[pair[1]]
                else:
                    links_f[pair[0]]['end_points'] = [links_f[pair[1]]['end_points'][0],links_f[pair[0]]['end_points'][1]]
                    links_f[pair[0]]['points'] = links_f[pair[1]]['points'][:-1] + links_f[pair[0]]['points']
                    links_f[pair[0]]['objs'] = links_f[pair[1]]['objs'][:-1] + links_f[pair[0]]['objs']
                    del links_f[pair[1]]
            else:
                if links_f[pair[1]]['end_points'][0] == i:
                    links_f[pair[0]]['end_points'] = [links_f[pair[0]]['end_points'][0],links_f[pair[1]]['end_points'][1]]
                    links_f[pair[0]]['points'] = links_f[pair[0]]['points'][:-1] + links_f[pair[1]]['points']
                    links_f[pair[0]]['objs'] = links_f[pair[0]]['objs'][:-1] + links_f[pair[1]]['objs']
                    del links_f[pair[1]]
                else:
                    links_f[pair[0]]['end_points'] = [links_f[pair[0]]['end_points'][0],links_f[pair[1]]['end_points'][0]]
                    links_f[pair[0]]['points'] = links_f[pair[0]]['points'][:-1] + links_f[pair[1]]['points'][::-1]
                    links_f[pair[0]]['objs'] = links_f[pair[0]]['objs'][:-1] + links_f[pair[1]]['objs'][::-1]
                    del links_f[pair[1]]
        elif len(pair) == 1 and len(links_f[pair[0]]['points']) <4:
            del links_f[pair[0]]
        elif len(pair)>2:
            print(pair)
    nodes = array([links_f[j]['end_points'][0] for j in links_f]+[links_f[j]['end_points'][1] for j in links_f])
    n_list = array(list(set(nodes)))
#     co = collections.Counter(nodes)
#     degrees = array([co[node] for node in n_list])
    degrees = array([len(where(nodes==i)[0]) for i in n_list])
#         ends = n_list[where(degrees==1)[0]]
    targets = n_list[where(degrees==2)[0]]              


        

In [3]:
net = {'links':NR}
edge_list = [net['links'][i]['end_points'] for i in net['links']]
adj_list = co.defaultdict(list)
for i,j in edge_list:
    adj_list[i].append(j)
    adj_list[j].append(i)

for i in adj_list:
    adj_list[i] = list(set(adj_list[i]))

In [ ]:
network = gln.simplify_v8(adj_list,net,core=95)
network.find_base_loops(1)
network.get_min_loop_set()
network.separate_base_loops()
network.cal_normalization_const_parallel()
norm = network.new_norm
print(norm)
network.cal_m2_old_parallel()
network.cal_m2_normalized_new(norm = norm)
print(network.m2_normalized)